In [ ]:
from openai import OpenAI
from dotenv import load_dotenv,find_dotenv

_ : bool = load_dotenv(find_dotenv())

client : OpenAI = OpenAI()

In [ ]:
import json

def weather(location:str, unit:str="fahrenheit")->str:
    
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [ ]:
from openai.types.chat.chat_completion import ChatCompletion,ChatCompletionMessage
import json

def conversation(prompt : str)->str:
    messages = [{"role": "user","content":prompt}]
    tools = [
        {
            "type" : "function",
            "function":{
                "name": "weather",
                "description": "Get weather of given location",
                "parameters":{
                    "type":"object",
                    "properties":{
                        "location":{
                            "type": "string",
                            "description": "The city, e.g Tokyo"
                        },
                        "unit": {
                            "type": "string", "enum": ["celsius", "fahrenheit"]
                        },
                    },
                    "required": ["location"],
                }

            }
        }
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ", list(tool_calls))

    if tool_calls:
        available_functions:{
            "weather": weather
        }

        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

            display("* Second Request Messages: ", list(messages))
            second_response: ChatCompletion = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=messages,
            )
            print("* Second Response: ", dict(second_response))
            return second_response.choices[0].message.content
        
conversation("What's the weather in Tokyo")